In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import pickle
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

import time 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
import scipy.io as sio
import copy
import pickle
import numpy as np
from scipy.signal import find_peaks
from os.path import dirname, join as pjoin
import datetime
import csv
import math
import sys

output_directory = "./turb_PP_fcn"

In [2]:
# Import Data 
with open('/content/drive/MyDrive/NSF_Research/NN/Data/fDOM_data', 'rb') as f: fDOM_data = pickle.load(f)
with open('/content/drive/MyDrive/NSF_Research/NN/Data/turb_data', 'rb') as f: turb_data = pickle.load(f)
with open('/content/drive/MyDrive/NSF_Research/NN/Data/stage_data', 'rb') as f: stage_data = pickle.load(f)

In [87]:
print(X.shape)

(1047, 3, 61)


In [ ]:
X,y = create_turbPP_dataset(fDOM_data, stage_data, turb_data, window_size = 30)
X = np.array(X)
y = np.array(y)

num_classes = len(np.unique(y, axis = 0))

# transform labels from integers to one hot vectors
encoder = OneHotEncoder(categories='auto')
encoder.fit(y.reshape(-1,1))
y = encoder.transform(y.reshape(-1,1)).toarray()
y_true = np.argmax(y, axis =1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

fcn = FCN(output_directory, input_shape, num_classes)
fcn.fit(X_train, y_train, X_test, y_test, y_true)

(837, 3, 61)


In [24]:
class FCN:

  def __init__(self, output_directory, input_shape, nb_classes):
    self.output_directory = output_directory
    self.model = self.build_model(input_shape, nb_classes)
    print(self.model.summary())
    self.model.save_weights(self.output_directory+'model_init.hdf5')
    return

  def build_model(self, input_shape, nb_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    overfit_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience= 5,verbose = True)

    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), metrics=['accuracy'])

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, 
      min_lr=0.0001)

    file_path = self.output_directory+'best_model.hdf5'

    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', 
      save_best_only=True)

    self.callbacks = [reduce_lr,model_checkpoint, overfit_callback]

    return model 

  def fit(self, x_train, y_train, x_val, y_val,y_true):
    
    # x_val and y_val are only used to monitor the test loss and NOT for training  
    batch_size = 16
    num_epochs = 50

    mini_batch_size = int(min(x_train.shape[0]/10, batch_size))

    start_time = time.time() 

    hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=num_epochs,
      verbose=True, validation_data=(x_val,y_val), callbacks=self.callbacks)
    
    duration = time.time() - start_time

    self.model.save(self.output_directory+'last_model.hdf5')

    model = keras.models.load_model(self.output_directory+'best_model.hdf5')

    y_pred = model.predict(x_val)

    # convert the predicted from binary to integer 
    y_pred = np.argmax(y_pred , axis=1)

    # save_logs(self.output_directory, hist, y_pred, y_true, duration)

    # keras.backend.clear_session()

  def predict(self, x_test, y_true,x_train,y_train,y_test):
    model_path = self.output_directory + 'best_model.hdf5'
    model = keras.models.load_model(model_path)
    y_pred = model.predict(x_test)
    return np.argmax(y_pred, axis=1)
		

In [38]:
def create_turbPP_dataset(fDOM_data, stage_data, turb_data, window_size = 15):
    
    # Get turb candidate peaks - these were the values used to get the set of turb candidate peaks
    params = {'prom' : [6,None],
                    'width': [None, None],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .6}
    
    turb_peaks, turb_props = find_peaks(turb_data[:,1], 
                                        prominence = params['prom'],
                                        width = params['width'],
                                        wlen = params['wlen'],
                                        distance = params['dist'],
                                        rel_height = params['rel_h'])
    
    turb_peaks, turb_props = delete_missing_data_peaks(turb_data, turb_peaks, turb_props, '/content/drive/MyDrive/NSF_Research/NN/Data/flat_plat_ranges.txt')

    # Import ground truths for these peaks
    gt_fname_t = '/content/drive/MyDrive/NSF_Research/NN/Data/turb_pp_0k-300k-2_labeled'
    with open(gt_fname_t, 'r', newline = '') as f:
        reader = csv.reader(f, delimiter = ',')
        # gt entries in form: ['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak']
        next(reader)
        ground_truth = [0 if row[2] == 'NPP' else 1 for row in reader] 
        f.close()  

    # Reshape data 
    fDOM_data = fDOM_data[:,1].reshape(-1,1)
    stage_data = stage_data[:,1].reshape(-1,1)
    turb_data = turb_data[:,1].reshape(-1,1)
                                         
        
    # Use Robust scaler to scale data
    fDOM_scaler = RobustScaler().fit(fDOM_data)
    fDOM_data_scaled = fDOM_scaler.transform(fDOM_data)
    
    turb_scaler = RobustScaler().fit(turb_data)
    turb_data_scaled = turb_scaler.transform(turb_data)
    
    stage_scaler = RobustScaler().fit(stage_data)
    stage_data_scaled = stage_scaler.transform(stage_data)
    
    # Created "sequenced" data, where middle point is the peak --> sequence is (window_size * 2) + 1
    X = []
    y = []
    
    for i,peak in enumerate(turb_peaks):
        if peak - window_size > 0 and peak + window_size < len(fDOM_data): 
            sample = np.hstack((fDOM_data_scaled[peak - window_size:peak + window_size + 1],
                                stage_data_scaled[peak - window_size:peak + window_size + 1],
                                turb_data_scaled[peak - window_size:peak + window_size + 1]))
            y.append(ground_truth[i])
            X.append(sample)
    
    return X, y
X,y = create_turbPP_dataset(fDOM_data, stage_data, turb_data, window_size = 30)
# X = np.array(X)
# y = np.array(y)
# print(X.shape)
# print(X[0])

In [37]:
f = fDOM_data[0:30,1].reshape(-1,1)
t = turb_data[0:30,1].reshape(-1,1)
s = stage_data[0:30,1].reshape(-1,1)

x = np.hstack((f,t,s))
print(x.shape)
print(x[0:5])

(30, 3)
[[23.41625856  0.37038     0.414     ]
 [23.35176495  0.34945476  0.414     ]
 [23.17670763  1.00651259  0.414     ]
 [23.21663052  1.01488318  0.414     ]
 [23.17056495  0.59637476  0.414     ]]


In [4]:
def datetime_to_julian(date : datetime.datetime) -> float:
    """
    Convert datetime object to julian time using algorithm outlined by Fliegel and van Flandern (1968):
    
    date:   date to convert

    return: julian time equivalent
    """
    interm1 = math.floor((14-date.month)/12)
    interm2 = date.year + 4800 - interm1
    interm3 = date.month + 12*interm1 - 3

    jdn = (date.day + math.floor((153*interm3 + 2)/5) + 365*interm2 + 
           math.floor(interm2/4) - math.floor(interm2/100) + math.floor(interm2/400) - 32045)

    jd = jdn + (date.hour - 12) / 24 + date.minute / 1440 + date.second / 86400 + date.microsecond / 86400000000
    return jd

def delete_missing_data_peaks(data, peaks, props, missing_file_path):
    """ 
    Delete peaks that occur during time periods designated as "missing data"
    
    data:              timeseries that peaks occured in
    peaks:             indices of peaks detected
    props:             properties associated with each peak
    missing_file_path: file path of missing date ranges
    return:            filtered peaks and props
    """
    with open(missing_file_path,newline='') as file:
        reader = csv.reader(file, delimiter = ',')
        time_list = []
        for row in reader:
            time_list.append([datetime_to_julian(datetime.datetime.strptime(row[0],'%Y-%m-%d %H:%M:%S')),
                              datetime_to_julian(datetime.datetime.strptime(row[1],'%Y-%m-%d %H:%M:%S'))])
            
        # Identify and remove violating peaks 
        keep_indices = list(np.linspace(0,peaks.shape[0]-1,peaks.shape[0]))
        for i,idx in enumerate(peaks): 
            time = data[idx,0]
            for row in time_list: 
                if time >= row[0] and time <= row[1]:
                    keep_indices.remove(i)
                    break
        
        peaks = np.take(peaks,keep_indices,0)  
        
        # Remove properties for violating peaks
        for key in props:
            props[key] = np.take(props[key], keep_indices,0)
        
        return peaks, props

In [81]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))